In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime


import keras.backend as K
from keras import metrics

from keras.layers import Input, Dense, add, Activation, SimpleRNN, TimeDistributed
from keras.models import Model, Sequential
from tensorflow import set_random_seed
from numpy.random import seed

# [PLACEHOLDER TITLE]

\[DRAFT\]<br>
Several police departments across the Unites States have been experimenting with software for crime prdiction. This  started a controversial debate: Critics are questioning the predictiv power of such models and point out biases towards certain crime typs and neighborhoods \[SOURCE\]. We took this as occacion to look into the publicly available crime records of the city of chicago. The data set contains filed complains over multiple police departments from DATE TO DATE. An examplary of the raw data is displayed. Columns including textual discriptions have been ommited. 

After briefly exploring the geo-spatial data, it became obvious that geo cordinates have not been recorded precisly enough to identify the exact location of the crime scnes. Instead we will focus on temporal patterns in the data set. For this we count the number of records per day and department. This results 

A model predicting the number of filed complains per department and day may be used to faciliate personel planning across different departments. We take this aplication as an example to give an comprehensive introduction of Recurrent Neural Networks with Long Short Term Memory and Gated Recurrent Cell in Keras. We will first introduce the basic concept behind RNNs and show how they may be trained on different input formats. We will then introduce RNN with more complex cell structures: 
[Placeholder outline]




## Introduction to Recurrent Neural Networks
<p style='text-align: justify;'>
Many machine learning tasks are based on the assumption of identically independent distributed (i.i.d.) samples. Thus a prediction for a label $y_i$ only depends on the corresponding feature vector $\vec{x_i}$, where $i$ denotes the index of the sample. When working with sequential data, one typically wants to predict a future outcome $\vec{y_{t+1}}$, which often equals the following observable vector $\vec{x_{t+1}}$, from a sequence of previously observed vectors $\vec{x_t}$, …, $\vec{x_{t-T}}$. $t$ denotes the last observed point of time and $T$  the number of observations since the start of the sequence.
While the vectors within a sequence are assumed to be dependent, we might still work with multiple mutually independent time series in a dataset. In this case one could use de notation $\vec{x_{ti}}$, corresponding to the observed vector at time $t$ from the sample sequence $i$. For simplicity we omit the sample index below. Note that many sequential prediction tasks are based on one dimensional time series, in that case one may also omit the vector notation. However, we stick to the more general vector notation. <br>
A simple feedforward network may be used to make predictions under the i.i.d assumption by passing one feature vector $\vec{x_i}$ at a time through the network. However, if one wants to account for temporal dependencies between the observed vectors $\vec{x_t}$, …, $\vec{x_{t-T}}$ the network needs some kind of memory. That’s the concept behind recurrent neural networks(RNNs): They use hidden activations, referred to as hidden state, to carry information within the network from one observation to the following one.
<table align="right" bgcolor="white">
    <tr><td bgcolor="white"><img align="right" src="RNN_t2.png" alt="NN" style="width: 500px;"/></td> </tr>
    <tr><td><p style='text-align: left;'> Image 1: Recurrent Neural Network with a sequence length of two.</p></td></tr>
</table>
<p style='text-align: justify;'>
Image 1 illustrates how a simple RNN may be used to predict $\vec{x_{t+1}}$ given a sequence of two previous observed vectors $\vec{x_t}$ and $\vec{x_{t-1}}$. The rectangles represent the input vectors. Each circle represents a vector of hidden activations and the triangle represents the output i.e. the predicted label. The arrows illustrate layer operations, i.e. the multiplication of the previous vector with one of the weight matrices $W_{in}$, $W_h$, or $W_{out}$, denoted by different colours. To make a prediction for $\vec{x_{t+1}}$, the first vector in the sequence $\vec{x_{t-1}}$ gets multiplied with the input weights $W_{in}$ and the activation function is applied to produce the first hidden state $\vec{h_{t-1}}$. Since $\vec{x_{t-1}}$ is the first observation in the sequence, $\vec{h_{t-1}}$ represents all available information at the time step $t-1$. To carry this information to the next hidden state $\vec{h_t}$, $\vec{h_{t-1}}$ gets multiplied with the hidden weight matrix $W_h$. Consecutively the new input $\vec{x_t}$ is multiplied with the input matrix $W_{in}$. The resulting vector carries the new information extracted from $\vec{x_t}$. This newly available information needs to be combined with the knowledge about previous observations to create $\vec{h_t}$. This can be done by simply adding up the two outputs from the matrix products and applying the  hyperbolic tangents as activation function to the vector sum. This is illustrated by the intersect of the green and the orange arrow. Conceptually the weights in $W_h$ represent the importance of previous observations and the weights in $W_{in}$ represent the importance of new information to the prediction task.
All available information at the time step $t$ is now aggregated in $\vec{h_t}$. To make a prediction for $\vec{x_{t+1}}$ it gets multiplied with the output weight matrix $W_{out}$. The size of $W_{out}$ and the corresponding activation function are, as usually, defined by the prediction task (regression or classification). This steps cover the forward pass in a simple RNN.
If we want to look back more steps in time when making a prediction for $\vec{x_{t+1}}$, we just need to add another hidden state and another input for every additional time step. Image 3 illustrates how this may look for a sequence of length three. Interestingly, even though RNNs can get quite deep along the time dimension, they have relatively few parameters. This is the case because all inputs or hidden states represent the same thing, another time step in the series or another aggregation of past observations. Thus, we only have to train three different weight matrices, one for the input, one for the hidden state and one for the output.<br>
Drawing RNNs with more time steps becomes cumbersome. We have to keep adding inputs and hidden states. The same holds true if we would like to construct them in in python. A more elegant and flexible representation is given in Image 4. Compared to Image 3 the repetitive elements have been put into a loop. For every time step in the sequence length $T$, the new input is feed into the network and combined with the previous hidden state. The new hidden state is passed back to the start of the loop as input to the next iteration. The loop stops at the end of the sequence and the last hidden state is put through the output layer. To include the first timestep within the loop representation, an initial hidden state $\vec{h_0}$ is defined as a vector of zeros. \[Howard, 2018\]
</p>
<table>
    <tr>
        <td bgcolor="white"><img  src="RNN_t3.png" alt="NN" style="width: 550px;"/></td>
        <td bgcolor="white"><img  src="RNN_Loop.png" alt="NN" style="width: 350px;"/></td>
    </tr>
    <tr>
        <td><p style='text-align: left;'> Image 2: Recurrent Neural Network with a sequence length of three. </p></td>
        <td><p style='text-align: left;'> Image 3: Recurrent Neural Network with a loop and a sequence length of $T$</p></td>
    </tr>
</table>

<p style='text-align: justify;'>
Keras provides a predefined model layer that constructs the described RNN behind the scenes. However, to emphasise the described operations, we demonstrate how to construct it by yourself using fully connected layers and the keras functional API. In keras a layer refers to a set of layer operations, typically a matrix multiplication and the application of an activation function, that output a new vector of activations. We follow this notation below.<br>
The functional  API allows to define a list of inputs and put them through several layers to produce a (list of) output(s). It then connects inputs and output(s) so the keras backend can be used to derive the gradients and train the model. This is more flexible than keras' Sequential model, which is a bit easier to use, but does not allow to pass multiple inputs through different layers in parallel. We start implementing the RNN by initializing the input list. The basic properties of a simple RNN are defined by the number of the hidden activations H_SZ , the sequence length SEQ_LEN and the number of features FEAT_SZ per time step. By choosing h_sz, the shape of the three weight matrices is sufficiently defined: The second and first dimension of $W_{in}$ and $W_{out}$ are respectively defined by the shape of the input data and the labels. Moreover, the first dimension of $W_{in}$ must match the first dimension of $W_h$ so their outputs can be added together. $W_h$ must be squared because the result of the matrix product $W_h$ x $h_t$ gets again multiplied with $W_h$. Since ht gets also multiplied by $W_{out}$ the second dimension of $W_{out}$ must also be H_SZ. We set H_SZ to five and SEQ_LEN to ten after little manual tuning. In general, the parameters did not seem to have a great impact on the model performance. Since the crime data consists of one-dimensional time series FEAT_SZ is given as one. \[Howard, 2018\]
</p>

In [2]:
seed(1)
set_random_seed(2)

# create list to keep track of inputs
input_list = []

# initialize RNN properties
H_SZ = 5
SEQ_LEN = 10
FEAT_SZ = 1

<p style='text-align: justify;'>
The Dense layer in keras holds the weights of a fully connected layer. It is initialized by passing the output dimension to the Dense() class. The input dimension will be interfered from the first input. After initialization, it can be called on a tensor which will return the resulting output activations. The three layers of the network are initialized by passing the respective number of output activations H_SZ, H_SZ and FEAT_SZ to the Dense() class.
</p>

In [3]:
# initilize fully connected layers and the activation function of the hidden state
first_layer = Dense(H_SZ, name = 'first_layer', use_bias=True)
hidden_layer = Dense(H_SZ , name = 'hidden_layer', use_bias=True)
tanh = Activation('tanh')
output_layer = Dense(FEAT_SZ, activation='relu', name = 'output_layer', use_bias=True)

<p style='text-align: justify;'>
We  loop through the sequence length and create for every time step a new input x and append it to the input list. The input activations act_in are calculated for every time step by putting the input variable x through the first layer. We additionally initialize h0 as a vector of zeros with the same (variable) shape as the input activations at the first time step. The new hidden state ht gets updated implementing the formula described above. After the loop the final hidden state is put through the output layer to make a prediction. Finally the functional API Model() connects input and output variables. Since keras typically works with tensors a function that splits a three-dimensional tensor along the second dimension, which corresponds to time dimension in keras, is defined.
</p>

In [4]:
for i in range(SEQ_LEN):
  
  # get input vector and append it to input list
  x = Input(shape = (FEAT_SZ,))
  input_list.append(x)
  
  # calculate input actiavtions
  act_in = first_layer(x)
  
  if(i == 0):
  # initialize h0 with zeros and append it to input list
    h0 = K.zeros_like(act_in)
    input_list.append(h0)  
    ht = Input(tensor = h0, name = 'h0')
  
  # calculate hidden activations
  ht = tanh(add([hidden_layer(ht), act_in])) 
  
# calculate output
out = output_layer(ht)

myRNN = Model(inputs=input_list, outputs=out)

In [5]:
def get_input(X):
  # slice 3-dim tensor along 2nd axis into list of inputs
  return([X[:,i,:] for i in range(X.shape[1])])

## Formatting the input data
<p style='text-align: justify;'>
<table align="right" bgcolor="white"> <tr><td bgcolor="white"><img align="right" src="Format1.png" alt="NN" style="width: 500px;"/></td> </tr>
    <tr><td><p style='text-align: left;'> Image4: Formatting data input for recurrent neural networks.</p></td></tr>
</table></p>
<p style='text-align: justify;'>
After counting the reported crimes per day and district we have a matrix of 22 x 6358 referring to districts by time steps. By training one model on 22 time series, we assume each series to be a different realization of the same underlying sequential pattern. An obvious alternative would be to train a separate model for each district. 
Before further formatting we split the data into train, validation and test set. The validation and test set should resemble the data, the model would see in production. Therefore, the most recent year is divided into half to create a validation and a test set. Since the data contains positive discrete counts, the minimal value is per definition zero. We will therefore normalize the input data by dividing the counts through the maximal value in the training set.<br>
An RNN cannot be trained on 22 long time series samples. Reasons for this are the small sample size, general difficulties of training deep networks \[Nielsen, 2018\], and the vanishing gradient problem explained below. Instead it is common practice to split long time series into subsequences, treating them as independent series. The simplest way to do this is, to slide a window of length SEQ_LEN through each series and treat each subseries contained in the window as an independent sample. The direct successor after the window defines the respective label for the subseries (Image 4). \[Fischer & Krauss, 2018\]
This is implemented by looping through the 22 full sequences and using array slicing withn a list comprehension to create the sub-sequences. The resulting sub-sequences are stacked vertically. Recurrent layers in keras and MyRNN above expect an input tensor of the dimensions \[batch_size, timesteps, input_dim\]. Since we work with one dimensional series, we have ignored the third dimension so far. In order to match the required input shape a thrid dimension of size one is added to the array.

</p>

In [6]:
# load data
FILE = "full_seqs.npy"
full_seqs = np.load(FILE)

# split into train, validation and test data 
full_seqs_tr = full_seqs[:, :-366]
full_seqs_val = full_seqs[:,-366 :-183]
full_seqs_ts = full_seqs[:,-183:]

# get maximum of training set for normalizaition
MAX = full_seqs_tr.max()

full_seqs_tr.shape, full_seqs_val.shape, full_seqs_ts.shape

((22, 6174), (22, 183), (22, 183))

In [7]:
def cut_sequences(long_seqs, SEQ_LEN):
  # input matrix of long sequences [sequences, timesteps]
  # returns matrix of subsequences X [subsequences, SEQ_LEN, 1], and labels y[subsequences,1]
  X = []
  y = []

  for long_seq in long_seqs:
    n = long_seq.shape[0]
    # slide window of SEQ_LEN over full sequence
    seq_stacked = np.stack([long_seq[i:i+SEQ_LEN] for i in range(n-SEQ_LEN)])
    labels = np.array([long_seq[i] for i in np.arange(SEQ_LEN, n)]).reshape(-1,1)
    X.append(seq_stacked)
    y.append(labels)

  X =  np.vstack(X)
  #add axis for number of features per time step = 1  
  X = X[:,:,np.newaxis]
  y =  np.vstack(y)
  
  return(X,y)

In [8]:
# cut sequences in sub sequences of length SEQ_LEN
X_tr, y_tr = cut_sequences(full_seqs_tr, SEQ_LEN)
X_val, y_val = cut_sequences(full_seqs_val, SEQ_LEN)
X_ts, y_ts = cut_sequences(full_seqs_ts, SEQ_LEN)

# normalize data
X_tr = X_tr/MAX
X_val = X_val/ MAX
X_ts = X_ts/ MAX

X_tr.shape, y_tr.shape, X_val.shape, y_val.shape

((135608, 10, 1), (135608, 1), (3806, 10, 1), (3806, 1))

## Training a simple Recurrent Neural Network
### Evaluation Metric and Benchmark
<p style='text-align: justify;'>
Before training and testing a model one should consider how the prediction error may be evaluated w.r.t. the application objective. We use the mean squared error (MSE) as loss function, which is common for regression tasks. 
MSE is hard for users to interpret in the application context of the model. The mean absolute error (MAE) seems more interpretable: In combination with average crime counts it provides some intuition about the uncertainty included in model predictions. To get a basic feeling for model performance while building the model we establish as naive benchmark: We calculate the MAE for predicting the last observed crime count for the succeeding crime count. This results in a MAE of 6.85 which does not seem to bad for such a simple benchmark. Given the mean of 47 crimes per day the naive prediction is on average by about 15 percent off.
<p>

In [9]:
# naive benchmark: predict last observed count
AVG = y_tr.mean().round(3)
MAE = np.abs(y_val[1:] - y_val[:-1] ).mean().round(3)

print("Average daily crime count by district: " + str(int(AVG)))
print("MAE predict last value: " + str(MAE))

Average daily crime count by district: 47
MAE predict last value: 6.845


### Training
<p style='text-align: justify;'>
Like other neural networks, RNNs are trained trough back propagation. To derive the gradient for an RNN one must calculate the impact on the loss considering all inputs back to the first time step. This is called backpropagation through time (bptt). We look in more detail into the derivation when introducing the vanishing gradient problem and deriving the gradient for RNNs with Long-Short-Term Memory. Here we use the keras .compile method to pass the loss function, the optimizer and MAE as additional evaluation metric. <br>
We give subsequently a overview over different RNN implementations and their predictive behaviour. To keep the complexity to a minimum while focusing on the theoretical concepts, we use a fixed hyperparameterset and only evaluate model performance briefly based on the MAE results on the validation set. The three introduced models all converge quickly so we train them all for 50 epochs. Since the performance differences are subtle we evaluate the model based on the average MAEs over the last 10 epochs. This way we want to reduce the effects of prediction varaiance at the end of difference epochs. Later we look look more rigorously into validation and training loss and compare models after proper parameter tuning. We choose a batchsize of 30 as before by some manuall testing.
</p>

In [10]:
B_SZ = 30
EP = 50
myRNN.compile(loss='mean_squared_error', optimizer='adam', metrics = ['mae'])

In [11]:
s = datetime.now()
myRNN_hist = myRNN.fit(get_input(X_tr), y_tr, batch_size = B_SZ, epochs = EP , validation_data = (get_input(X_val), y_val), verbose = 0)
e = datetime.now()
myRNN_time = round((e - s).total_seconds(),1)
myRNN_tail_mae = np.mean(myRNN_hist.history['val_mean_absolute_error'][-10:]).round(3)
print('myRNN training time: ' + str(myRNN_time))
print('myRNN tail epochs MAE: ' + str(myRNN_tail_mae))

myRNN training time: 341.2
myRNN tail epochs MAE: 5.492


After fitting myRNN for 50 epochs the average MAE over the last epochs is about 5.50. While this is a notable improvement compared to the benchmark one may expect a greter improvment the additional model complexity. Whether this results would justify a practical application is questionable.

### Alternative formatting with prediction sequence

<p style='text-align: justify;'>
Before introducing advanced recurrent architectures, we take a second look at the input format. In terms of data usage the introduced format seems quite inefficient: By sliding a window one step at time through the sequences we create mostly redundant sample sequences. A more efficient way of presenting data to the model is illustrated in image 6. Instead of changing one value in each sample sequence, the sample sequences are now defined by disjunct sub-sequences of lenght SEQ_LEN. Instead of predicting one label after observing a complete sub-sequence, we assigne a corresponding label sequences of the same length to each sample sequence. Each label sequence starts one time step ahead of the sample sequence.
The remaining values, i.e. the remainder of the full sequence length divided by SEQ_LEN, are dropped. This reduced the size of the data set by a factor of SEQ_LEN, without losing any information compared to the previous formatting.
</p>

<table bgcolor="white"> <tr><td bgcolor="white"><img align="right" src="Format1.png" alt="NN" style="width: 500px;"/></td> </tr>
    <tr><td><p style='text-align: left;'> Image 4: Alternative input format for recurrent neural networks</p></td></tr>
</table>


In [12]:
def cut_sequence_return_state(long_seqs, SEQ_LEN, cut_seq_start = True):
  long_seqs_X = long_seqs[:,:-1]
  long_seqs_y = long_seqs[:,1:]
  if(cut_seq_start):
    start = long_seqs_X.shape[1] % SEQ_LEN
    X = [long_seqs_X[:,i:i+SEQ_LEN] for i in np.arange(start,long_seqs_X.shape[1],SEQ_LEN)]
    y = [long_seqs_y[:,i:i+SEQ_LEN] for i in np.arange(start,long_seqs_y.shape[1],SEQ_LEN)]
  else:
    start = 0
    X = [long_seqs_X[:,i:i+SEQ_LEN] for i in np.arange(start,long_seqs_X.shape[1],SEQ_LEN)]
    y = [long_seqs_y[:,i:i+SEQ_LEN] for i in np.arange(start,long_seqs_y.shape[1],SEQ_LEN)]
    if X[-1].shape[1] != SEQ_LEN:
      X.pop()
      y.pop()
    
  X = np.vstack(X)[:,:,np.newaxis]
  y = np.vstack(y)[:,:,np.newaxis]
  return X, y

X_tr, y_tr = cut_sequence_return_state(full_seqs_tr, SEQ_LEN)
X_val, y_val = cut_sequence_return_state(full_seqs_val, SEQ_LEN, cut_seq_start = False)
X_ts, y_ts = cut_sequence_return_state(full_seqs_ts, SEQ_LEN, cut_seq_start = False)

<p style='text-align: justify;'>
We now replace myRNN by the predefined SimpleRNN layer in combination with a fully connected output layer. While this implements the same behaviour as myRNN it allows for additional functionality. By setting the return_sequence parameter to True it passes all hidden states, not only the last one, to the following layer. In the myRRN code above, this could have been implemented, by appending all hidden states to a list instead of updating the variable ht. The hidden states within the list are then passed through the output layer to make for every time step in the input sequence a prediction the next time step in the label sequence. To implement this above, the output layer may be included in de loop (Compare Image 3). The predictions then need to be save in a new output list. In keras this can also can be done more conviniently, by wrapping the output layer in the TimeDistributed Wrapper. Image 9 shows the unrolled illustration of an RNN predicting sequences.
</p>
<table bgcolor="white"> <tr><td bgcolor="white"><img align="right" src="RNN_Ret_Seq.png" alt="NN" style="width: 500px;"/></td> </tr>
    <tr><td><p style='text-align: left;'> Image 4: Recurrent Neural Network with return sequence</p></td></tr>
</table>


In [14]:
# simple RNN with return sequence
model = Sequential()
model.add(SimpleRNN(H_SZ, input_shape=(SEQ_LEN,FEAT_SZ), return_sequences=True) )
model.add(TimeDistributed(Dense(FEAT_SZ)))
model.compile(loss='mean_squared_error', optimizer='adam', metrics = ['mae'])

In [15]:
s = datetime.now()
seqRNN_hist = model.fit(X_tr, y_tr, batch_size = B_SZ,  epochs = EP, validation_data = (X_val, y_val), verbose = 0)
e = datetime.now()
seqRNN_time = round((e - s).total_seconds(),1)
seqRNN_tail_MAE = np.mean(seqRNN_hist.history['val_mean_absolute_error'][-10:]).round(3)
print('Return sequence training time: ' + str(seqRNN_time))
print('Return sequence tail epochs MAE: ' + str(seqRNN_tail_MAE))

Return sequence training time: 66.3
Return sequence tail epochs MAE: 5.898


<p style='text-align: justify;'>
Training the model on the new data format is clearly faster due to the reduced sample size.
On the small crime data set this may not have a big impact. However on bigger datasets with a more complex architecture, this may be a relevant factor to consider when building your RNN.
On the downside the MAE on the validation set increases by about 0.40. The reason for this becomes clear by looking at Image 9. When making the first prediction for $x_{t-3}$ there is no information available on the previous hidden states. Thus, the predictions at the beginning of each sub-sequence get worse, increasing the average prediction error. Annother interesting application of return sequences is the possibility to stack multiple RNN cells vertically. This allows to pass the hidden state sequence as input to annother RNN layer, creating a more abstract representation of the input sequence. We are not going to look into this, further information can be found at \[source\].
</p>

### Stateful RNNs
<p style='text-align: justify;'>
The problem of missing information occurs, because the full sequences are cut into sub-sequences. If the available information from the preceeding sub-sequence could be passed on to the the following one, this might be avoided.  In fact, this can be done by saving the values from the last hidden states and using them as initialization of the $h_{t0}$ vector (See Image3). This behaviour is choosen by setting the stateful parameter of a RNN layer to true. To keep track of sub-sequences that belong to the same full sequence the batch size has to be fixed to the number of full sequences in the data set, here the number of districts. Note that the sample sequences in the new format are ordered by time, so the first row of the input tensor holds the first sub-sequence of the first district. The second row holds the first sub-sequence of the second district and so on(Image 5). This is relevant because, a batch is passed through the network in parallel. By setting a batch size to the number of districts, keras passes the last hidden state of each row, holding a sub-sequence of a certain district, to the same row of the following batch.
The order of the data set has been ignored so far, because keras shuffles the rows by default at the beginning of every epoch. This can be turned off by setting the shuffle parameter of the .fit method to False. After one epoch, i.e. after the full sequences of all districts have been passed through the network, the last states need to be reset to zero. Therefore the "stateful RNN" needs to be trained in a loop to manually reset the state at the end of the epoch.
</p>

In [16]:
# stateful RNN
B_SZ = 22

model = Sequential()
model.add(SimpleRNN(H_SZ, batch_input_shape=(B_SZ, SEQ_LEN, FEAT_SZ), return_sequences=True, stateful=True))
model.add(TimeDistributed(Dense(FEAT_SZ)))
model.compile(loss='mean_squared_error', optimizer='adam', metrics = ['mae'])

In [17]:
stateful_val_mae = []
s = datetime.now()
for i in range(EP):
    stateRNN_hist = model.fit(X_tr, y_tr, batch_size = B_SZ, epochs = 1, validation_data = (X_val, y_val), verbose = 0, shuffle=False)
    model.reset_states()
    stateful_val_mae.append(stateRNN_hist.history['val_mean_absolute_error'])
e = datetime.now()
stateRNN_time = round((e - s).total_seconds(),1)
stateRNN_tail_MAE = np.mean(stateful_val_mae[-10:]).round(3)
print('Stateful training time: ' + str(stateRNN_time))
print('Stateful tail epochs MAE: ' + str(stateRNN_tail_MAE))

Stateful training time: 122.7
Stateful tail epochs MAE: 5.473


<p style='text-align: justify;'>
The expression “statefull RNN” is misleading since RNNs are stateful by design. So stateful in keras actually means stateful between batches.[Source] 
It is important to understand that the value of the last state is simply copied to the initial hidden state of the next subsequence. This may improve the prediction error, but when deriving the gradient, they are treated as constant. The inputs considered when updating the weights are still restricted to (sub-)sequence length. Therefore, practitioners argue that the “unstateful” RNNs achieve in many application better results[SOURCE]. Reasons for this are, the loss of randomness by stopping to shuffle the data between epochs and the fixed batch size, which is a parameter to which RNNs might be sensitive.
[Source] gives as a rule of thumb to use “stateful RNNs” when the output mainly depends on the occurence of a certain input. This may be the case in many natural language tasks. If the outputs represent a complex function of the previous time steps increasing the subsequence length may be necessary instead. <br>
Indeed, the stateful model reaches with 5.47 about the same MAE as myRNN, yet it did barely improve it. 
Since we do not see an improvment and the statefullness comes with several restircitions like the fixed batch size we will continue to work on the orginal data format. 

So far we worked with a small sequence length of ten and did not elaborate why we introduce the restrictive "statefull RNN" instead of increasing the sequence length. The reason for this is the vanishing gradient problem, which makes it difficult to train simple RNNs on long sequences. We will explain this below and continue hot to overcome ths restriction with more complex RNNs.
</p>
